In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from GenZ import chunked_moddeling
from GenZ import get_model_df, System, get_summary_table, simplify_df, get_configs, ParallelismConfig, get_runtime_breakdown
from GenZ.Models.get_language_model import create_full_chunked_model, create_full_decode_model
from GenZ.Models.attention import mha_flash_attention_chunked
from GenZ.utils.plot_rooflines import display_df
import os
import pandas as pd


In [3]:
get_configs('meta-llama/meta-llama-3.1-405b')

In [4]:
import ipywidgets as widgets
from ipywidgets import interact

# def interactive_chunked_model_runtime(input_tokens, output_tokens, output_batch):
input_tokens = 1024
output_tokens = 1024
TP = 4
chunk_size = 512
system = System(frequency=1000 , flops=4500, off_chip_mem_size=(192*1024), compute_efficiency=1, memory_efficiency=1,
                offchip_mem_bw=8000, bits='int8', external_mem_bw=128, interchip_link_bw=900, interchip_link_latency=0.25, num_nodes=4)
runtime_plot_df = pd.DataFrame(columns=['Batching Type', 'Layer Name', 'Runtime', 'Chunk Size', 'Decode Batch'])
for chunk_size in [512,  2048]:
    for output_batch in [1, 16, 64]:
        # model = create_full_decode_model(name='gpt-3', input_sequence_length = output_tokens, tensor_parallel=TP)
        # model_df = get_model_df(model, system, batch_size=output_batch,  model_characterstics = False)
        # summary_table = get_summary_table(model_df, model_characterstics = False)
        # runtime_df = get_runtime_breakdown(model_df)
        # # display(model_df)
        # # display(summary_table)
        # df = pd.DataFrame(
        #     [   ['Continous', 'Embedding',   runtime_df.Embedding, chunk_size, output_batch],
        #         ['Continous', 'Collective',  runtime_df.Collective, chunk_size, output_batch],
        #         ['Continous', 'LA_layers',   runtime_df.LA_layers, chunk_size, output_batch],
        #         ['Continous', 'QKVO_layers', runtime_df.QKVO_layers, chunk_size, output_batch],
        #         ['Continous', 'FFN_layers',  runtime_df.FFN_layers, chunk_size, output_batch]],
        #     columns=['Batching Type', 'Layer Name', 'Runtime','Chunk Size', 'Decode Batch'])
        # runtime_plot_df = pd.concat([runtime_plot_df, df])
        # model = create_full_chunked_model(name='gpt-3',
        #                                     prefill_kv_sizes= [(input_tokens, chunk_size-output_batch)],
        #                                     decode_kv_sizes = [output_tokens] * output_batch, tensor_parallel=TP)
        # model_df = get_model_df(model, system, batch_size=1,  model_characterstics = False)
        # summary_table = get_summary_table(model_df, model_characterstics = False)
        # runtime_df = get_runtime_breakdown(model_df)
        # display(summary_table)
        # display(model_df)
        chunked_output = chunked_moddeling(model='gpt-3', 
                        prefill_kv_sizes = [(input_tokens, chunk_size-output_batch)],
                        system_name=system, 
                        decode_kv_sizes = [output_tokens] * output_batch, tensor_parallel=TP)
        runtime_df = chunked_output['Runtime_breakdown']
        df = pd.DataFrame(
                [['Chunked', 'Embedding',   runtime_df.Embedding, chunk_size, output_batch],
                ['Chunked', 'Collective',  runtime_df.Collective, chunk_size, output_batch],
                ['Chunked', 'LA',   runtime_df.LA_layers, chunk_size, output_batch], 
                ['Chunked', 'QKVO', runtime_df.QKVO_layers, chunk_size, output_batch],
                ['Chunked', 'FFN',  runtime_df.FFN_layers, chunk_size, output_batch]],
            columns=['Batching Type', 'Layer Name', 'Runtime','Chunk Size', 'Decode Batch'])
        runtime_plot_df = pd.concat([runtime_plot_df, df])


    # return fig
    # display(runtime_plot_df)

# input_tokens_slider = widgets.BoundedIntText(min=1, max=20480, step=10, value=1024, description='Input Tokens:')
# output_tokens_slider = widgets.BoundedIntText(min=1, max=20480, step=10, value=1024, description='Output Tokens:')
# batch_slider = widgets.BoundedIntText(min=1, max=512, step=8, value=8, description='Output Batch:')

# interactive_chunked_model_runtime(input_tokens=1024, output_tokens=1024, output_batch=8)

/var/folders/h1/r_4mhdb92_7cg5n015yr61wh0000gp/T/ipykernel_7798/4116165449.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  runtime_plot_df = pd.concat([runtime_plot_df, df])


In [ ]:
import plotly.express as px
runtime_plot_df['Decode Batch'] = runtime_plot_df['Decode Batch'].astype(str)



fig = px.bar(runtime_plot_df, x='Decode Batch', y='Runtime', color='Layer Name', facet_row='Chunk Size',
            pattern_shape="Batching Type", barmode='relative',
            title='Runtime Breakdown by Chunk Size, Input Tokens = {}, Output Tokens = {}'.format(input_tokens, output_tokens))

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
)
fig.show()

In [ ]:
runtime_plot_df

In [ ]:
import matplotlib.pyplot as plt

runtime_plot_df['Decode Batch'] = runtime_plot_df['Decode Batch'].astype(str)
runtime_plot_df['Decode Batch'] = pd.Categorical(runtime_plot_df['Decode Batch'], categories=runtime_plot_df['Decode Batch'].unique(), ordered=True)

fig, ax = plt.subplots(figsize=(10, 4), ncols=2)



for i, (key, grp) in enumerate(runtime_plot_df[runtime_plot_df['Batching Type'] == 'Chunked'].groupby(['Chunk Size'])):
    # print(grp.pivot(index=['Decode Batch'], columns='Layer Name', values='Runtime'))
    grp.pivot(index=['Decode Batch'], columns='Layer Name', values='Runtime').plot(kind='bar', stacked=True, ax=ax[i], label=key)

    # ax[i].set_title('Runtime Breakdown by Chunk Size, Input Tokens = {}, Output Tokens = {}, Chunk Size= {}'.format(input_tokens, output_tokens, key[0]))
    ax[i].set_title('Chunk Size= {}'.format(key[0]))
    ax[i].set_ylabel('Latency (ms)')
    ax[i].xaxis.set_tick_params(rotation=0)
    ax[i].tick_params(axis='both', which='major', labelsize=24)

    ax[i].title.set_size(30)
    ax[i].xaxis.label.set_size(24)
    ax[i].yaxis.label.set_size(24)
    # ax[i].xaxis.tick_size = 32
    ax[i].legend(loc='upper left', fontsize=18)
    # ax[i].legend = 2
    ax[i].set_ylim(0, 50)
    if i == 1:
        ax[i].legend().set_visible(False)
        ax[i].set_ylabel('')
    ax[i].set_xlabel('Batch')
plt.savefig('chunked_runtime_breakdown.pdf', format='pdf', dpi=300)
# plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
system = System(frequency=1000 , flops=4500, off_chip_mem_size=(192*1024), compute_efficiency=1, memory_efficiency=1,
                offchip_mem_bw=8000, bits='int8', external_mem_bw=128, interchip_link_bw=900, interchip_link_latency=0.25, num_nodes=4)
chunked_output = chunked_moddeling(model='deepseek-ai/DeepSeek-V3-Base',
                        prefill_kv_sizes = [(0, 256)],
                        system_name=system, debug=True, bits='int8',
                        decode_kv_sizes = [256, 512] , tensor_parallel=8)

In [ ]:
system = System(frequency=1000 , flops=4500, off_chip_mem_size=(192*1024), compute_efficiency=1, memory_efficiency=1,
                offchip_mem_bw=8000, bits='int8', external_mem_bw=128, interchip_link_bw=900, interchip_link_latency=0.25, num_nodes=4)
chunked_output = chunked_moddeling(model='deepseek-ai/DeepSeek-V3-Base',
                        prefill_kv_sizes = [], model_profilling=True,
                        system_name=system, debug=True, bits='int8',
                        decode_kv_sizes = [1] , tensor_parallel=1)
display(chunked_output[0])
display(chunked_output[1])


In [5]:
chunked_moddeling(model='gpt-3', 
                        prefill_kv_sizes = [],
                        decode_kv_sizes= [],
                        system_name=system,
                        tensor_parallel=TP, debug=True)

/opt/homebrew/Caskroom/miniconda/base/envs/genz_pip_test/lib/python3.12/site-packages/GenZ/utils/plot_rooflines.py:58: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, '% of total time'] = 100*df.loc[i,'Cycles']/total_cycles
/opt/homebrew/Caskroom/miniconda/base/envs/genz_pip_test/lib/python3.12/site-packages/GenZ/utils/plot_rooflines.py:58: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, '% of total time'] = 100*df.loc[i,'Cycles']/total_cycles
/opt/homebrew/Caskroom/miniconda/base/envs/genz_pip_test/lib/python3.12/site-packages/GenZ/utils/plot_rooflines.py:58: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, '% of total time'] = 100*df.loc[i,'Cycles']/total_cycles
/opt/homebrew/Caskroom/miniconda/base/envs/genz_pip_test/lib/python3.12/site-packages/GenZ/utils/plot_rooflines.py:58: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, '% of total time'] = 100*df.loc[i,'Cycles']/total_cycles
/opt/homebre

,Layer Name,Op Type,Dimension,Op Intensity,Latency (msec),Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Compute time (msec),Memory time (msec),Communication time (msec),Bound,C/M ratio,Cycles,% of total time,Throughput (Tflops),Compute cycle,Memory cycle,C Effcy,Communication cycle
0,embeddings,GEMM,"[((1, 12568, 0), (12288, 12568), (1, 12288, 0))]",0.000000,0.000000,0.000000,0.000000,294.562500,0.000000,294.562500,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
1,Emb_AR,Sync,"(1, 0, 12288)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
2,QKV,GEMM,"[((1, 12288, 0), (9216, 12288), (1, 9216, 0))]",0.000000,0.000000,0.000000,0.000000,20736.000000,0.000000,20736.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
3,Out Proj,GEMM,"[((1, 3072, 0), (12288, 3072), (1, 12288, 0))]",0.000000,0.000000,0.000000,0.000000,6912.000000,0.000000,6912.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
4,MHA AR,Sync,"(1, 0, 12288)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
5,up+gate,GEMM,"[((1, 12288, 0), (12288, 12288), (1, 12288, 0))]",0.000000,0.000000,0.000000,0.000000,27648.000000,0.000000,27648.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
6,down,GEMM,"[((1, 12288, 0), (12288, 12288), (1, 12288, 0))]",0.000000,0.000000,0.000000,0.000000,27648.000000,0.000000,27648.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
7,FFN AR,Sync,"(1, 0, 12288)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
8,classifier,GEMM,"[((1, 12288, 0), (12568, 12288), (1, 12568, 0))]",0.000000,0.000000,0.000000,0.000000,294.562500,0.000000,294.562500,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000
9,classifier_AG,Sync,"(1, 0, 50272)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0,0.000000,nan,0.000000,0.000000,0.000000,1,0.000000


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,0.00000,83533.12500,83533.12500,83533.12500,0,294.56250,0.00000,0.00000,0,0.00000,0.00000


/opt/homebrew/Caskroom/miniconda/base/envs/genz_pip_test/lib/python3.12/site-packages/GenZ/LLM_inference/llm_chunked.py:98: RuntimeWarning: invalid value encountered in scalar divide
  thrpt = 1000 * (sum([i[1] for i in prefill_kv_sizes]) + len(decode_kv_sizes)) / chunked_latency


{'Latency': 0.0,
 'Throughput': nan,
 'Runtime_breakdown': Embedding: 0.0, MHA: 0.0, FFN: 0.0, Collective: 0.0, LA_layers: 0, QKVO_layers: 0.0, FFN_layers: 0.0, Softmax: 0, AR_time: 0.0, A2A_time: 0, Send_Recv_time: 0, Mamba_time: 0,
 'is_offload': False,
 'model_df':        Layer Name    Op Type  \
 0      embeddings       GEMM   
 1          Emb_AR       Sync   
 2          Repeat     Repeat   
 3             QKV       GEMM   
 4        Out Proj       GEMM   
 5          MHA AR       Sync   
 6         up+gate       GEMM   
 7            down       GEMM   
 8          FFN AR       Sync   
 9      End Repeat  EndRepeat   
 10     classifier       GEMM   
 11  classifier_AG       Sync   
 
                                            Dimension       Bound C/M ratio  \
 0   [((1, 12568, 0), (12288, 12568), (1, 12288, 0))]  Collective         0   
 1                                      (1, 0, 12288)  Collective         0   
 2                                                 96  Collectiv

In [16]:
chunked_moddeling(model='meta-llama/meta-llama-3.1-70b',
                        prefill_kv_sizes = [],
                        decode_kv_sizes= [(1,8,2589, 30)],
                        system_name='H100_GPU', system_eff=0.75,
                        tensor_parallel=4, debug=True)['Latency']

,Layer Name,Op Type,Dimension,Op Intensity,Latency (msec),Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Compute time (msec),Memory time (msec),Communication time (msec),Bound,C/M ratio,Cycles,% of total time,Throughput (Tflops),Compute cycle,Memory cycle,C Effcy,Communication cycle
0,embeddings,GEMM,"[((1, 8000, 0), (8192, 8000), (1, 8192, 0))]",0.000000,0.000000,0.000000,0.000000,125.000000,0.000000,125.000000,0.000000,0.000000,0.000000,Collective,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000
1,Emb_AR,Sync,"(1, 0, 8192)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000
2,QKV,GEMM,"[((1, 8192, 8), (2560, 8192), (1, 2560, 8))]",7.967322,1.226440,26843.545600,10.000000,3200.000000,3.125000,3213.125000,0.036189,1.226440,0.000000,Memory,0.029508,1226439.631842,8.469188,21.887376,36189.478396,1226439.631842,0.750000,0.000000
3,Logit Pre,Logit,"((1, 16, 1, 128), (1, 2, 2589, 128), (1, 16, 1, 2589))",7.507578,0.039210,848.363520,0.312500,101.132812,6.320801,107.766113,0.001144,0.039210,0.000000,Memory,0.029169,39210.236410,0.270767,21.636277,1143.732417,39210.236410,0.750000,0.000000
4,Attend Pre,Attend,"((1, 16, 1, 2589), (1, 2, 2589, 128), (1, 16, 1, 128))",7.507578,0.039210,848.363520,6.320801,101.132812,0.312500,107.766113,0.001144,0.039210,0.000000,Memory,0.029169,39210.236410,0.270767,21.636277,1143.732417,39210.236410,0.750000,0.000000
5,Logit Suf,Logit,"((1, 128, 1, 128), (1, 2, 30, 128), (1, 128, 1, 30))",17.614679,0.000672,78.643200,2.500000,1.171875,0.585938,4.257812,0.000106,0.000672,0.000000,Memory,0.157769,672.018346,0.004641,117.025377,106.023862,672.018346,0.750000,0.000000
6,Attend Suf,Attend,"((1, 128, 1, 30), (1, 2, 30, 128), (1, 128, 1, 128))",17.614679,0.000672,78.643200,0.585938,1.171875,2.500000,4.257812,0.000106,0.000672,0.000000,Memory,0.157769,672.018346,0.004641,117.025377,106.023862,672.018346,0.750000,0.000000
7,Out Proj,GEMM,"[((1, 2048, 8), (8192, 2048), (1, 8192, 8))]",7.961127,0.981296,21474.836480,2.500000,2560.000000,10.000000,2572.500000,0.028952,0.981296,0.000000,Memory,0.029503,981296.381400,6.776350,21.884149,28951.582717,981296.381400,0.750000,0.000000
8,MHA AR,Sync,"(1, 8, 8192)",0.000000,0.912552,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.912552,Collective,0.000000,912552.083333,6.301635,0.000000,0.000000,0.000000,0.750000,912552.083333
9,up+gate,GEMM,"[((1, 8192, 8), (14336, 8192), (1, 14336, 8))]",7.987742,6.873277,150323.855360,10.000000,17920.000000,17.500000,17947.500000,0.202661,6.873277,0.000000,Memory,0.029485,6873276.654412,47.463465,21.870770,202661.079016,6873276.654412,0.750000,0.000000


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,276706.75456,33186.16992,32890.00000,125.00000,204.60938,224.34375,14.48119,14481194.55830,0.07976,12.56753,1.83390


14.481194558303597

In [18]:
chunked_moddeling(model='meta-llama/meta-llama-3.1-70b',
                        prefill_kv_sizes = [],
                        decode_kv_sizes= [(8,1621)], bits='int8',
                        system_name='H100_GPU', system_eff=0.75,
                        tensor_parallel=2, debug=True)['Latency']

,Layer Name,Op Type,Dimension,Op Intensity,Latency (msec),Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Compute time (msec),Memory time (msec),Communication time (msec),Bound,C/M ratio,Cycles,% of total time,Throughput (Tflops),Compute cycle,Memory cycle,C Effcy,Communication cycle
0,embeddings,GEMM,"[((1, 16000, 0), (8192, 16000), (1, 8192, 0))]",0.000000,0.000000,0.000000,0.000000,125.000000,0.000000,125.000000,0.000000,0.000000,0.000000,Collective,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000
1,Emb_AR,Sync,"(1, 0, 8192)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Collective,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000
2,QKV,GEMM,"[((1, 8192, 8), (5120, 8192), (1, 5120, 8))]",15.959478,1.226078,53687.091200,5.000000,3200.000000,3.125000,3208.125000,0.036189,1.226078,0.000000,Memory,0.029516,1226077.942028,8.693212,43.787666,36189.478396,1226077.942028,0.750000,0.000000
3,Logit Dec,Logit,"((1, 256, 1, 128), (1, 32, 1621, 128), (1, 256, 1, 1621))",14.989200,0.196467,8498.708480,2.500000,506.562500,31.660156,540.722656,0.005729,0.196467,0.000000,Memory,0.029159,196467.131571,1.393003,43.257661,5728.822703,196467.131571,0.750000,0.000000
4,Attend Dec,Attend,"((1, 256, 1, 1621), (1, 32, 1621, 128), (1, 256, 1, 128))",14.989200,0.196467,8498.708480,31.660156,506.562500,2.500000,540.722656,0.005729,0.196467,0.000000,Memory,0.029159,196467.131571,1.393003,43.257661,5728.822703,196467.131571,0.750000,0.000000
5,Out Proj,GEMM,"[((1, 4096, 8), (8192, 4096), (1, 8192, 8))]",15.953262,0.980935,42949.672960,2.500000,2560.000000,5.000000,2567.500000,0.028952,0.980935,0.000000,Memory,0.029514,980934.691585,6.955082,43.784437,28951.582717,980934.691585,0.750000,0.000000
6,MHA AR,Sync,"(1, 8, 8192)",0.000000,0.570851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.570851,Collective,0.000000,570850.694444,4.047480,0.000000,0.000000,0.000000,0.750000,570850.694444
7,up+gate,GEMM,"[((1, 8192, 8), (28672, 8192), (1, 28672, 8))]",15.979936,6.871362,300647.710720,5.000000,17920.000000,17.500000,17942.500000,0.202661,6.871362,0.000000,Memory,0.029494,6871361.825980,48.719743,43.753730,202661.079016,6871361.825980,0.750000,0.000000
8,down,GEMM,"[((1, 14336, 8), (8192, 14336), (1, 8192, 8))]",15.975484,3.436638,150323.855360,8.750000,8960.000000,5.000000,8973.750000,0.101331,3.436638,0.000000,Memory,0.029485,3436638.327206,24.366660,43.741541,101330.539508,3436638.327206,0.750000,0.000000
9,FFN AR,Sync,"(1, 8, 8192)",0.000000,0.570851,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.570851,Collective,0.000000,570850.694444,4.047480,0.000000,0.000000,0.000000,0.750000,570850.694444


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,566702.89920,34023.50488,32890.00000,125.00000,1013.12500,224.28125,14.10385,14103854.74324,0.39293,12.56295,1.14797


14.103854743242783

In [7]:
chunked_moddeling(model='meta-llama/meta-llama-3.1-70b',
                        prefill_kv_sizes = [(491,222)],
                        decode_kv_sizes= ([(1, 64, 491, 205)]),
                        system_name='H100_GPU', system_eff=0.75,
                        tensor_parallel=2, debug=False)['Latency']

29.641686757405047

In [11]:
chunked_moddeling(model='meta-llama/meta-llama-3.1-70b',
                        prefill_kv_sizes = [(491,222)],
                        decode_kv_sizes= ([(64, 491+ 205)]),
                        system_name='H100_GPU', system_eff=0.75,
                        tensor_parallel=2, debug=True)['Latency']

,Layer Name,Op Type,Dimension,Op Intensity,Latency (msec),Num ops (MFLOP),Input_a (MB),Input_w (MB),Output (MB),Total Data (MB),Compute time (msec),Memory time (msec),Communication time (msec),Bound,C/M ratio,Cycles,% of total time,Throughput (Tflops),Compute cycle,Memory cycle,C Effcy,Communication cycle
0,embeddings,GEMM,"[((1, 16000, 222), (8192, 16000), (1, 8192, 222))]",213.261686,0.099664,58195.968000,6.774902,250.000000,3.468750,260.243652,0.078458,0.099664,0.000000,Memory,0.787219,99664.388918,0.308680,583.919378,78457.658241,99664.388918,0.750000,0.000000
1,Emb_AR,Sync,"(1, 222, 8192)",0.000000,0.014528,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014528,Collective,0.000000,14527.669271,0.044995,0.000000,0.000000,0.000000,0.750000,14527.669271
2,QKV,GEMM,"[((1, 8192, 286), (5120, 8192), (1, 5120, 286))]",262.199740,2.587548,1919313.510400,357.500000,6400.000000,223.437500,6980.937500,2.587548,2.493004,0.000000,Compute,1.037924,2587547.705292,8.014149,741.750000,2587547.705292,2493004.227516,0.750000,0.000000
3,Logit Pre,Logit,"((1, 32, 222, 128), (1, 4, 713, 128), (1, 32, 222, 713))",102.269487,0.139851,103734.312960,138.750000,55.703125,772.880859,967.333984,0.139851,0.130378,0.000000,Compute,1.072660,139850.775814,0.433146,741.750000,139850.775814,130377.501444,0.750000,0.000000
4,Attend Pre,Attend,"((1, 32, 222, 713), (1, 4, 713, 128), (1, 32, 222, 128))",102.269487,0.139851,103734.312960,772.880859,55.703125,138.750000,967.333984,0.139851,0.130378,0.000000,Compute,1.072660,139850.775814,0.433146,741.750000,139850.775814,130377.501444,0.750000,0.000000
5,Logit Dec,Logit,"((1, 2048, 1, 128), (1, 256, 696, 128), (1, 2048, 1, 696))",7.448829,1.351348,29192.355840,40.000000,3480.000000,217.500000,3737.500000,0.039356,1.351348,0.000000,Memory,0.029124,1351347.613698,4.185392,21.602403,39356.057755,1351347.613698,0.750000,0.000000
6,Attend Dec,Attend,"((1, 2048, 1, 696), (1, 256, 696, 128), (1, 2048, 1, 128))",7.448829,1.351348,29192.355840,217.500000,3480.000000,40.000000,3737.500000,0.039356,1.351348,0.000000,Memory,0.029124,1351347.613698,4.185392,21.602403,39356.057755,1351347.613698,0.750000,0.000000
7,Out Proj,GEMM,"[((1, 4096, 286), (8192, 4096), (1, 8192, 286))]",258.885304,2.070038,1535450.808320,178.750000,5120.000000,357.500000,5656.250000,2.070038,1.999576,0.000000,Compute,1.035239,2070038.164233,6.411319,741.750000,2070038.164233,1999575.546364,0.750000,0.000000
8,MHA AR,Sync,"(1, 286, 8192)",0.000000,1.335825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.335825,Collective,0.000000,1335824.652778,4.137314,0.000000,0.000000,0.000000,0.750000,1335824.652778
9,up+gate,GEMM,"[((1, 8192, 286), (28672, 8192), (1, 28672, 286))]",273.713809,14.490267,10748155.658240,357.500000,35840.000000,1251.250000,37448.750000,14.490267,14.341586,0.000000,Compute,1.010367,14490267.149633,44.879235,741.750000,14490267.149633,14341586.243873,0.750000,0.000000


,MACs (MFLOP),Total Data (MB),Total Weights (MB),Unused Weights (MB),KV Cache (MB),On-chip Memory Footprint (MB),Latency (msec),Cycles,Attn Latency (msec),Linear Latency (msec),Comm Latency (msec)
0,19976020.29568,78922.17090,65780.00000,0,7071.40625,468.10938,32.28724,32287241.85054,2.98240,26.59373,2.71112


32.287241850542834